In [ ]:
import cv2
import pandas as pd
import keras
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Input, Dense, GlobalAveragePooling2D
from keras import backend as K
import os
import numpy as np
from numpy.random import seed
import json
from collections import Counter
from keras.optimizers import SGD
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import Image as Image2
from IPython.display import display
from matplotlib.pyplot import imshow
import urllib
import random
import tensorflow as tf
from scipy.spatial import distance_matrix
from glob import glob
from tqdm import tqdm

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
images_glob = glob('../input/data/images_*/images/*.png')
print(f'Number of images {len(images_glob)}')
xray_data = pd.read_csv('../input/data/Data_Entry_2017.csv')
image_paths = { os.path.basename(path): path for path in images_glob }
xray_data['full_path'] = xray_data['Image Index'].map(image_paths.get)
nih_paths = list(xray_data['full_path'])

In [ ]:
train_paths = list(glob('../input/ranzcr-clip-catheter-line-classification/train/*.jpg'))
test_paths = list(glob('../input/ranzcr-clip-catheter-line-classification/test/*.jpg'))
sample_images = list(nih_paths + train_paths +test_paths)
sample_images =sample_images[:50000]
print(f'length of nih paths is {len(nih_paths)}')
print(f'length of train paths is {len(train_paths)}')
print(f'length of test paths is {len(test_paths)}')
print(f'length of all images is {len(sample_images)}')

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42)

In [ ]:
# trainable = False is import because we'll be using this system for features but not output

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(1024, 1024, 3))
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
pool_2d = GlobalAveragePooling2D(name='pool_2d')(base_model.output)
dense1 = Dense(1024, name='dense_1', activation='relu')(pool_2d)
dense2 = Dense(2048, name='dense_2', activation='relu')(dense1)
predictions = Dense(1024, activation='relu', name='dense_3')(dense2)
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
output_list = np.arange(0,1024 ).reshape(1, 1024)
for c in tqdm(sample_images):
    use_images = cv2.imread(c)
    use_images =cv2.resize(use_images, (1024, 1024))
    use_images = use_images.reshape((1,1024, 1024, 3))
    model_output = model.predict(use_images, batch_size=1, verbose=0)
    output_list =np.concatenate((output_list, model_output), axis=0) 

In [ ]:
output_list= output_list[1:]
output_list.shape

In [ ]:
df1 = pd.DataFrame(output_list, index = sample_images)


In [ ]:
df1.to_csv('from0to50k.csv')